In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
import pandas as pd
import numpy as np

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('paper')
sns.set_style('white')

In [5]:
import sys
sys.path.append('/home/dan/bclm')

In [7]:
spdf = pd.read_csv('../data/spdf_fixed.csv.gz', low_memory=False)
spdf.head()

,id,form,lemma,upostag,xpostag,feats,token_id,sent_id,token_str,global_sent_id,...,deps,misc,ner_escaped,set,duplicate_sent_id,very_similar_sent_id,biose_layer0,biose_layer1,biose_layer2,biose_layer3
0,1,עשרות,עשר,CDT,CDT,gen=F|num=P,1,1,עשרות,1,...,_,_,_,dev,NaN,NaN,O,O,O,O
1,2,אנשים,איש,NN,NN,gen=M|num=P,2,1,אנשים,1,...,_,_,_,dev,NaN,NaN,O,O,O,O
2,3,מגיעים,הגיע,BN,BN,gen=M|num=P|per=A|HebBinyan=HIFIL,3,1,מגיעים,1,...,_,_,_,dev,NaN,NaN,O,O,O,O
3,4,מ,מ,PREPOSITION,PREPOSITION,_,4,1,מתאילנד,1,...,_,_,_,dev,NaN,NaN,O,O,O,O
4,5,תאילנד,תאילנד,NNP,NNP,_,4,1,מתאילנד,1,...,_,_,GPE,dev,NaN,NaN,S-GPE,O,O,O


In [19]:
spdf.head().T

,0,1,2,3,4
id,1,2,3,4,5
form,עשרות,אנשים,מגיעים,מ,תאילנד
lemma,עשר,איש,הגיע,מ,תאילנד
upostag,CDT,NN,BN,PREPOSITION,NNP
xpostag,CDT,NN,BN,PREPOSITION,NNP
feats,gen=F|num=P,gen=M|num=P,gen=M|num=P|per=A|HebBinyan=HIFIL,_,_
token_id,1,2,3,4,4
sent_id,1,1,1,1,1
token_str,עשרות,אנשים,מגיעים,מתאילנד,מתאילנד
global_sent_id,1,1,1,1,1


In [17]:
from collections import OrderedDict
DEFAULT_FIELDS = ['id', 'form', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel', 'deps', 'misc']
def sentence_to_conllu_token_list(sent, add_misc=['ner_escaped', 'token_id', 'token_str', 'biose_layer0', 
                                                  'biose_layer1', 'biose_layer2', 'biose_layer3'],
                                  metadata=['sent_id', 'global_sent_id', 'set', 'duplicate_sent_id', 'very_similar_sent_id']):
    metadata = OrderedDict([(m, str(sent[m].iat[0])) for m in metadata])
    
    if add_misc is not None:
        amisc = sent.apply(lambda x: [m+'='+str(x[m]) for m in add_misc], axis=1)
        amisc = amisc.str.join('|')
    token_morphs = (sent.groupby('token_id').id.agg([min, max])
                 .pipe(lambda x: x[x['max'] - x['min']>0])
                 .assign(token_morphs=lambda x: x['min'].astype(str)+'-'+x['max'].astype(str))
                 .set_index('min')['token_morphs'])   
    token_list = []
    
    for (i, row), token_str, am in zip(sent[DEFAULT_FIELDS].iterrows(), sent['token_str'].tolist(), amisc.tolist()):
        token = OrderedDict(row)
        if type(token['feats']) == str and token['feats']!='_':
            #token['feats'] = OrderedDict([f.split('=') for f in token['feats'].split('|')])
            feats = OrderedDict()
            for f in token['feats'].split('|'):
                k,v = f.split('=')
                if k not in feats:
                    feats[k] = v
                else:
                    feats[k] = feats[k]+','+v
            token['feats'] = feats
                        
        am = OrderedDict([f.split('=') for f in am.split('|') ])
        if token['misc'] =='_' or token['misc'] is None:
            token['misc'] = am
        elif type(token['misc'])==str:
            token['misc'] = OrderedDict([('MISC', token['misc'])])
            token['misc'].update(am)
        elif type(token['misc'])==OrderedDict:
            am.update(token['misc'])
            token['misc'] = am
            
        if token['id'] in token_morphs:
            tok_row = OrderedDict([(f, '_') for f in DEFAULT_FIELDS])
            id_tup = token_morphs[token['id']].split('-')
            tok_row['id'] = (id_tup[0], '-', id_tup[1])
            tok_row['form'] = token_str
            token_list.append(tok_row)
            
        token_list.append(token)
        
    return token_list, metadata



In [18]:
from conllu.models import TokenList
sp_conllu_sents = [TokenList(*sentence_to_conllu_token_list(spdf[spdf.sent_id==i])) for i in spdf.sent_id.drop_duplicates().tolist()]
print(sp_conllu_sents[2].serialize())

# sent_id = 3
# global_sent_id = 3
# set = dev
# duplicate_sent_id = nan
# very_similar_sent_id = nan
1	יו"ר	יו"ר	NNT	NNT	gen=M|num=S	12	subj	_	biose=O|ner_escaped=_|token_id=1|token_str=יו"ר|biose_layer0=O|biose_layer1=O|biose_layer2=O|biose_layer3=O
2-3	הוועדה	_	_	_	_	_	_	_	_
2	ה	ה	DEF	DEF	_	3	def	_	biose=O|ner_escaped=_|token_id=2|token_str=הוועדה|biose_layer0=O|biose_layer1=O|biose_layer2=O|biose_layer3=O
3	וועדה	ועדה	NN	NN	gen=F|num=S	1	gobj	_	biose=O|ner_escaped=_|token_id=2|token_str=הוועדה|biose_layer0=O|biose_layer1=O|biose_layer2=O|biose_layer3=O
4	,	_	yyCM	yyCM	_	1	punct	_	biose=O|ner_escaped=_|token_id=3|token_str=,|biose_layer0=O|biose_layer1=O|biose_layer2=O|biose_layer3=O
5	ח"כ	ח"כ	NN	NN	gen=M|num=S	1	appos	_	biose=O|ner_escaped=_|token_id=4|token_str=ח"כ|biose_layer0=O|biose_layer1=O|biose_layer2=O|biose_layer3=O
6	אורה	אורה	NNP	NNP	_	5	nn	_	biose=B-PER|ner_escaped=PER[3]|token_id=5|token_str=אורה|biose_layer0=B-PER|biose_layer1=O|biose_layer2=O|biose_layer3=O
7	נמיר	נמ

In [29]:
for s in sp_conllu_sents:
    for t in s:
        if type(t['feats'])==OrderedDict:
            for k, v in t['feats'].items():
                if ',' in v:
                    print (k, v)

suf_gen F,M
gen F,M
gen F,M
num P,S
gen F,M
suf_gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
suf_gen F,M
suf_gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
suf_gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
suf_gen F,M
gen F,M
gen F,M
suf_gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
suf_gen F,M
suf_gen F,M
gen F,M
suf_gen F,M
gen F,M
gen F,M
gen F,M
suf_gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
num P,S
gen F,M
gen F,M
num P,S
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
gen F,M
suf_gen F,M
gen F,M
num D,P
gen F,M
gen F,M
